## Running the VUS cooccurence workflow via WES on TopMed COPDGene data
Other than the COPDGene vcf file which was the target of the analysis, the files used in the example were uploaded to the BioDataCatalyst Seven Bridges server.

Prior to the workflow the COPDGene_phs000951_TOPMed_WGS_freeze.8.chr13.hg38.c1.vcf was filtered down to the range of BRCA2 by using BCFTools.

In [1]:
from fasp.workflow import sbWESClient
cl = sbWESClient('https://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/ga4gh/wes/v1', 'forei/fasp-vus',
                     '~/.keys/sbbdc_key.json', debug=True)

/Users/forei/imfenvs/fasp/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Setting up the WES run

In [2]:
params = {
    "project": "forei/fasp-vus",
    "inputs": {
        'gnomad_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617c5ce7e6261a31b6d12de3',
                        'name': 'gnomad_chr13_brca2.vcf',
                        'class': 'File'},

        'vcf_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617c77bce6261a31b6d12f0a',
                     'name': 'COPDGene_phs000951_TOPMed_WGS_freeze.8.chr13.hg38.c1.filtered.vcf',
                     'class': 'File'},

        'pathogenicity_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617c5ce6e6261a31b6d12ddc',
                               'name': 'clinvar_brca2.tsv',
                               'class': 'File'}
        }
    }



Now we have formulated the body in the way that it can be passed to a client function as follows.

In [3]:
import json
run_id= cl.runGenericWorkflow(
    workflow_url="sbg://forei/fasp-vus/cooccurrence/2",
    workflow_params = json.dumps(params),
    workflow_type = "CWL",
    workflow_type_version = "sbg:draft-2"
)
run_id

'b20b60f9-5125-4088-b61f-de791c9278b0'

In [9]:
import dateutil.parser
print(cl.getTaskStatus(run_id))
log = cl.GetRunLog(run_id)
if log['run_log']['start_time']:
    start = dateutil.parser.isoparse(log['run_log']['start_time'])
    end = dateutil.parser.isoparse(log['run_log']['end_time'])
    duration = end - start
    print(str(duration))

COMPLETE
0:18:29


In [10]:
cl.GetRunLog(run_id)

{'request': {'tags': {},
  'workflow_params': {'name': 'cooccurrence run - 11-01-21 01:44:19',
   'project': 'forei/fasp-vus',
   'inputs': {'p2': None,
    'save_files': None,
    'gene': None,
    'chromosome': None,
    'data_directory': None,
    'ensembl_release': None,
    'pathology_file': None,
    'gnomad_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617f3fc9fe2ce002310283b8',
     'name': 'gnomad_chr13_brca2.vcf',
     'class': 'File'},
    'hg_version': None,
    'vcf_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617f3fc9fe2ce002310283ba',
     'name': '_1_COPDGene_phs000951_TOPMed_WGS_freeze.8.chr13.hg38.c1.filtered.vcf',
     'class': 'File'},
    'phased': None,
    'pathogenicity_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617f4673fe2ce0023102842c',
     'name': 'clinvar_brca2.tsv',
     'class': 'File'}}},
  'workflow_type': 'CWL',
  'workflow_engine_params': {},
  'workflow_url': 'sbg://forei/fasp-vus/cooccu

## Getting the results - via DRS
Once the run is complete, further steps can use DRS to obtain the file output from the workflow.

In [11]:
runLog = cl.GetRunLog(run_id)
runLog['outputs']

{'vpi_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617f4ac9e6261a31b6d14b11',
  'name': '_2_BRCA2-vpi.json',
  'class': 'File'},
 'pathology_output': None,
 'all_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617f4ac9e6261a31b6d14b0f',
  'name': '_2_BRCA2-all.json',
  'class': 'File'},
 'out_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617f4ac9e6261a31b6d14b17',
  'name': '_3_BRCA2-cooccurrences.json',
  'class': 'File'},
 'ipv_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617f4ac9e6261a31b6d14b14',
  'name': '_2_BRCA2-ipv.json',
  'class': 'File'},
 'tout_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617f4ac9e6261a31b6d14b13',
  'name': '_2_BRCA2-tout.json',
  'class': 'File'}}

In [12]:
resultsDRSID = runLog['outputs']['out_file']['path']
resultsDRSID

'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617f4ac9e6261a31b6d14b17'

Use the BioDataCatalyst SB DRS Server to retrieve the results files

In [15]:
from fasp.loc import sbbdcDRSClient
drsClient = sbbdcDRSClient('~/.keys/sevenbridges_keys.json', 's3')

### DRS GetObject
Here's how we then get details of the file. Note that here only the id portion of the DRS URI is being passed. It is the job of a metaresolver to look at that URI and to determine where to send the id. As noted, we are passing up on the opportunity to use a metaresolver and putting in the id manually.

In [16]:
sbDRSID = resultsDRSID.split('/')[-1]
fileDetails = drsClient.getObject(sbDRSID)
fileDetails

{'id': '617f4ac9e6261a31b6d14b17',
 'name': '_3_BRCA2-cooccurrences.json',
 'size': 81104,
 'checksums': [{'type': 'etag',
   'checksum': 'cad46dcd72010874721bd86a5297c77c-1'}],
 'self_uri': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617f4ac9e6261a31b6d14b17',
 'created_time': '2021-11-01T02:02:49Z',
 'updated_time': '2021-11-01T02:02:49Z',
 'mime_type': 'application/json',
 'access_methods': [{'type': 's3',
   'region': 'us-east-1',
   'access_id': 'aws-us-east-1'}]}

In [17]:
url = drsClient.getAccessURL(sbDRSID,'s3')

### Downloading the file
Now we can use the url obtained to download the file. We'll create a small function to encapsulate the download.

In [18]:
import requests
import os
def download(url, file_path):
    with open(os.path.expanduser(file_path), "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [19]:
fullPath = fileDetails['name']
download(url, fullPath)


In [20]:
with open(fullPath) as json_file:
    data = json.load(json_file)
# delete the local copy of the file
#os.remove(fullPath)
# and look at the contents
# data

In [21]:
# flatten cooccurrence output
flat_vus = []
for k, v in data['cooccurring vus'].items():
    pathogenic_count = len(v['pathogenic variants'])
    ## this is a pythonic way of merging dicts - it is cryptic
    z = {**{"vus":k}, **v['likelihood data'], **v['allele frequencies'], **{"no_pathogenic_coocurrs":pathogenic_count}}
    flat_vus.append(z)

# turn the array of dicts into a data frame    
import pandas as pd
flat_df = pd.DataFrame(flat_vus)
flat_df

,vus,p1,p2,n,k,likelihood,maxPop,maxPopFreq,cohortFreq,no_pathogenic_coocurrs
0,"(13, 32315831, 'G', 'A')",0.000049,0.001,9345,1,0.002811,None,None,0.914742,1
1,"(13, 32318080, 'C', 'T')",0.000049,0.001,10195,2,0.025604,None,None,0.997944,1
2,"(13, 32318598, 'T', 'C')",0.000049,0.001,9047,1,0.003733,None,None,0.885572,1
3,"(13, 32319654, 'A', 'G')",0.000049,0.001,8119,1,0.009026,None,None,0.794734,1
4,"(13, 32321240, 'G', 'C')",0.000049,0.001,10196,2,0.025580,None,None,0.998042,1
5,"(13, 32325741, 'C', 'T')",0.000049,0.001,10194,2,0.025628,None,None,0.997847,1
6,"(13, 32331128, 'G', 'A')",0.000049,0.001,10194,2,0.025628,None,None,0.997847,1
7,"(13, 32333969, 'A', 'G')",0.000049,0.001,10194,2,0.025628,None,None,0.997847,1
8,"(13, 32338918, 'A', 'G')",0.000049,0.001,10194,2,0.025628,None,None,0.997847,1
9,"(13, 32340868, 'G', 'C')",0.000049,0.001,10193,2,0.025653,None,None,0.997749,1


In [22]:
# homozygous vus output
homozygous_vus = []
for k, v in data['homozygous vus'].items():
    ## this is a pythonic way of merging dicts - it is cryptic
    z = {**{"vus":k}, **v}
    homozygous_vus.append(z)

# turn the array of dicts into a data frame    
import pandas as pd
hz_df = pd.DataFrame(homozygous_vus)
hz_df

,vus,count,maxPop,maxPopFreq,cohortFreq
0,"(13, 32318080, 'C', 'T')",9765,None,None,0.955854
1,"(13, 32318598, 'T', 'C')",4653,None,None,0.455462
2,"(13, 32319654, 'A', 'G')",3060,None,None,0.299530
3,"(13, 32321240, 'G', 'C')",9766,None,None,0.955951
4,"(13, 32325741, 'C', 'T')",9762,None,None,0.955560
...,...,...,...,...,...
549,"(13, 32360422, 'C', 'T')",1,None,None,0.000098
550,"(13, 32378219, 'T', 'C')",1,None,None,0.000098
551,"(13, 32391658, 'A', 'T')",1,None,None,0.000098
552,"(13, 32365109, 'C', 'CTT')",1,None,None,0.000098


## To do
- Submit the pathogenicity file from the local system
- Either  access the gnomad file from Gnomad, or supply it from the local system

## Done
- Make the container available to other WES servers by adding the Docker container to Docker Hub instead of the Seven Bridges docker repository
